In [3]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import glob
import math
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter

In [4]:
basedir = os.getcwd()
datadir = basedir + '/data'

# reading the csv file with annotated image file names
train_cultivar= pd.read_csv(datadir + '/sorghum-id-fgvc-9/train_cultivar_mapping.csv')
train_cultivar.dropna(inplace=True)

# turning cultivar labels into strings
train_cultivar['cultivar']=train_cultivar['cultivar'].astype(str)
train_cultivar.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/maymap/data/sorghum-id-fgvc-9/train_cultivar_mapping.csv'

In [5]:
# creating list of unique cultivars
labels=list(np.unique(train_cultivar['cultivar']))

NameError: name 'train_cultivar' is not defined

In [6]:
# visualizing frequency of cultivars 

import seaborn as sns
a=pd.DataFrame({'cultivar':train_cultivar['cultivar']})
plt.figure(figsize=(30,10))
sns.histplot(a,x='cultivar')
plt.xticks(rotation=45)
plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [7]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=0.9, horizontal_flip=True, shear_range=0.2, validation_split = 0.2)
train_data=train_datagen.flow_from_dataframe(train_cultivar, datadir + "/sorghum-id-fgvc-9/train_images/",color_mode='rgb', batch_size=32, x_col='image', y_col='cultivar' , classes=labels , class_mode='categorical', target_size=(300, 300))

NameError: name 'train_cultivar' is not defined